In [1]:
import copy
import glob
import os

import cdflib
import numpy as np

import torch
import re
import collections
from torch._six import string_classes, int_classes
import cv2
import time
import math
import matplotlib.pyplot as plt
import torch.nn as nn

In [2]:
# Load Human3.6M Skeleton

train_2d_file = '/media/ivan/Ivan/Project Lab/THESIS_FALL_DETECTION/2D to 3D Pose/3d_pose_baseline_pytorch/data/train_2d.pth.tar'
train_3d_file = '/media/ivan/Ivan/Project Lab/THESIS_FALL_DETECTION/2D to 3D Pose/3d_pose_baseline_pytorch/data/train_3d.pth.tar'
train_2d = torch.load(train_2d_file)
train_3d = torch.load(train_3d_file)

In [101]:
train_2d.keys()

dict_keys([(1, 'Directions', 'Directions 1.54138969.h5'), (1, 'Directions', 'Directions 1.55011271.h5'), (1, 'Directions', 'Directions 1.58860488.h5'), (1, 'Directions', 'Directions 1.60457274.h5'), (1, 'Directions', 'Directions.54138969.h5'), (1, 'Directions', 'Directions.55011271.h5'), (1, 'Directions', 'Directions.58860488.h5'), (1, 'Directions', 'Directions.60457274.h5'), (1, 'Discussion', 'Discussion 1.54138969.h5'), (1, 'Discussion', 'Discussion 1.55011271.h5'), (1, 'Discussion', 'Discussion 1.58860488.h5'), (1, 'Discussion', 'Discussion 1.60457274.h5'), (1, 'Discussion', 'Discussion.54138969.h5'), (1, 'Discussion', 'Discussion.55011271.h5'), (1, 'Discussion', 'Discussion.58860488.h5'), (1, 'Discussion', 'Discussion.60457274.h5'), (1, 'Eating', 'Eating 2.54138969.h5'), (1, 'Eating', 'Eating 2.55011271.h5'), (1, 'Eating', 'Eating 2.58860488.h5'), (1, 'Eating', 'Eating 2.60457274.h5'), (1, 'Eating', 'Eating.54138969.h5'), (1, 'Eating', 'Eating.55011271.h5'), (1, 'Eating', 'Eating.5

In [73]:
train_2d[(1, 'Directions', 'Directions 1.54138969.h5')].shape

(1383, 32)

In [103]:
#Rename Unconsistent Filename

for key in train_2d.keys():
#     print(key)
    filename = os.path.splitext(key[2])[0]
    videopath = "../2D to 3D Pose/Dataset/Human3.6m/Videos/S"+str(key[0])+"/Videos/"
    for file in os.listdir(videopath):
#         print(file)
        if file.endswith(".mp4"):
            if file.find("TakingPhoto")>-1:
                print("Rename taking photo")
                os.rename(os.path.join(videopath, file), os.path.join(videopath, file.replace("TakingPhoto", "Photo")))
            if file.find("WalkingDog")>-1:
                print("Rename Walking Dog")
                os.rename(os.path.join(videopath, file), os.path.join(videopath, file.replace("WalkingDog", "WalkDog")))


# Alpha Pose Code

In [17]:
index_alphapose={
    # Use 17 skeleton point
    "Nose": 0,
    #"Neck": "-",
    "RShoulder": 6,
    "RElbow": 8,
    "RWrist": 10,
    "LShoulder": 5,
    "LElbow": 7,
    "LWrist": 9,
    "RHip": 12,
    "RKnee": 14,
    "RAnkle": 16,
    "LHip": 11,
    "LKnee": 13,
    "LAnkle": 15,
    "REye": 2,
    "LEye": 1,
    "REar": 4,
    "LEar": 3
}


def data_converter(data):
    data=data['keypoints']
    keypoints=[]
    kp_score=[]
    for a in range (0,len(data)):
        score=[]
        if ((a+3)%3==0):
            keypoints.append(data[a])
            keypoints.append(data[a+1])
        elif((a+1)%3==0):
            score=[data[a]]
            kp_score.append(score)

    return keypoints
    
    
def euclidean_dist(a, b):
    # This function calculates the euclidean distance between 2 point in 2-D coordinates
    # if one of two points is (0,0), dist = 0
    # a, b: input array with dimension: m, 2
    # m: number of samples
    # 2: x and y coordinate
    try:
        if (a.shape[1] == 2 and a.shape == b.shape):
            # check if element of a and b is (0,0)
            bol_a = (a[:,0] != 0).astype(int)
            bol_b = (b[:,0] != 0).astype(int)
            dist = np.linalg.norm(a-b, axis=1)
            return((dist*bol_a*bol_b).reshape(a.shape[0],1))
    except:
        print("[Error]: Check dimension of input vector")
        return 0


# Function to find next in alpha pose because alpha pose did not have neck

def find_neck(alpha_example):
    
    # 10 and 11 is x and y coordinate for left shoulder of alpha pose
    x1= alpha_example[:,10] 
    y1= alpha_example[:,11]
    
    
    # 12 and 13 is x and y coordinate for right shoulder of alpha pose
    x2= alpha_example[:,12]
    y2= alpha_example[:,13]
    
    # Result will be  neck coordinate(array([586.35131836, 472.55082703]), array([559.9307251 , 480.63751221])) ((x1,x2),(y1,y2))
    neck_= ((x1 + x2)/2, (y1 + y2)/2)
    
    result_neck=[]
    for a,b in zip(neck_[0],neck_[1]):
        result_neck.append([a,b])
    
    neck=np.asarray(result_neck)
    return neck

def norm_alpha(X):
    num_sample = X.shape[0]
    # Keypoints
    Nose = X[:,index_alphapose['Nose']*2:index_alphapose['Nose']*2+2]
    Neck = find_neck(X)
    RShoulder = X[:,index_alphapose['RShoulder']*2:index_alphapose['RShoulder']*2+2]
    RElbow = X[:,index_alphapose['RElbow']*2:index_alphapose['RElbow']*2+2]
    RWrist = X[:,index_alphapose['RWrist']*2:index_alphapose['RWrist']*2+2]
    LShoulder = X[:,index_alphapose['LShoulder']*2:index_alphapose['LShoulder']*2+2]
    LElbow = X[:,index_alphapose['LElbow']*2:index_alphapose['LElbow']*2+2]
    LWrist = X[:,index_alphapose['LWrist']*2:index_alphapose['LWrist']*2+2]
    RHip = X[:,index_alphapose['RHip']*2:index_alphapose['RHip']*2+2]
    RKnee = X[:,index_alphapose['RKnee']*2:index_alphapose['RKnee']*2+2]
    RAnkle = X[:,index_alphapose['RAnkle']*2:index_alphapose['RAnkle']*2+2]
    LHip = X[:,index_alphapose['LHip']*2:index_alphapose['LHip']*2+2]
    LKnee = X[:,index_alphapose['LKnee']*2:index_alphapose['LKnee']*2+2]
    LAnkle = X[:,index_alphapose['LAnkle']*2:index_alphapose['LAnkle']*2+2]
    REye = X[:,index_alphapose['REye']*2:index_alphapose['REye']*2+2]
    LEye = X[:,index_alphapose['LEye']*2:index_alphapose['LEye']*2+2]
    REar = X[:,index_alphapose['REar']*2:index_alphapose['REar']*2+2]
    LEar = X[:,index_alphapose['LEar']*2:index_alphapose['LEar']*2+2]

    # Length of head
    length_Neck_LEar = euclidean_dist(Neck, LEar)
    length_Neck_REar = euclidean_dist(Neck, REar)
    length_Neck_LEye = euclidean_dist(Neck, LEye)
    length_Neck_REye = euclidean_dist(Neck, REye)
    length_Nose_LEar = euclidean_dist(Nose, LEar)
    length_Nose_REar = euclidean_dist(Nose, REar)
    length_Nose_LEye = euclidean_dist(Nose, LEye)
    length_Nose_REye = euclidean_dist(Nose, REye)
    length_head      = np.maximum.reduce([length_Neck_LEar, length_Neck_REar, length_Neck_LEye, length_Neck_REye, \
                                 length_Nose_LEar, length_Nose_REar, length_Nose_LEye, length_Nose_REye])

    # Length of torso
    length_Neck_LHip = euclidean_dist(Neck, LHip)
    length_Neck_RHip = euclidean_dist(Neck, RHip)
    length_torso     = np.maximum(length_Neck_LHip, length_Neck_RHip)
    #length_torso     = np.sqrt(np.square(Neck[:,0:1]-(LHip[:,0:1]+RHip[:,0:1])/2) + np.square(Neck[:,1:2]-(LHip[:,1:2]+RHip[:,1:2])/2))

    # Length of right leg
    length_leg_right = euclidean_dist(RHip, RKnee) + euclidean_dist(RKnee, RAnkle)
  

    # Length of left leg
    length_leg_left = euclidean_dist(LHip, LKnee) + euclidean_dist(LKnee, LAnkle)
 

    # Length of leg
    length_leg = np.maximum(length_leg_right, length_leg_left)

    # Length of body
    length_body = length_head + length_torso + length_leg
    
    # Check all samples have length_body of 0
    length_chk = (length_body > 0).astype(int)
    
    # Check keypoints at origin
    keypoints_chk = (X > 0).astype(int)
    
    chk = length_chk * keypoints_chk
    
    # Set all length_body of 0 to 1 (to avoid division by 0)
    length_body[length_body == 0] = 1
    
    # The center of gravity
    # number of point OpenPose locates:
    num_pts = (X[:, 0::2] > 0).sum(1).reshape(num_sample,1)
    centr_x = X[:, 0::2].sum(1).reshape(num_sample,1) / num_pts

    centr_y = X[:, 1::2].sum(1).reshape(num_sample,1) / num_pts


    # The  coordinates  are  normalized relative to the length of the body and the center of gravity
    X_norm_x = (X[:, 0::2] - centr_x) / length_body
    X_norm_y = (X[:, 1::2] - centr_y) / length_body
    
    # Stack 1st element x and y together
    X_norm = np.column_stack((X_norm_x[:,:1], X_norm_y[:,:1]))
        
    for i in range(1, X.shape[1]//2):
        X_norm = np.column_stack((X_norm, X_norm_x[:,i:i+1], X_norm_y[:,i:i+1]))
    
    # Set all samples have length_body of 0 to origin (0, 0)
    X_norm = X_norm * chk
    
    return X_norm

def AlphaPose_Inference(videopath):
    args = opt
    args.dataset = 'coco'
    args.sp = True


    if not args.sp:
        torch.multiprocessing.set_start_method('forkserver', force=True)
        torch.multiprocessing.set_sharing_strategy('file_system')

    args.video=videopath
    args.outputpath = "examples/res/"+os.path.splitext(os.path.basename(videopath))[0]+"/"
    args.save_video = True
    
    videofile = args.video
    mode = args.mode
    if not os.path.exists(args.outputpath):
        os.mkdir(args.outputpath)
    
    if not len(videofile):
        raise IOError('Error: must contain --video')

    # Load input video
    data_loader = VideoLoader(videofile, batchSize=args.detbatch).start()
    (fourcc,fps,frameSize) = data_loader.videoinfo()

    # Load detection loader
    print('Loading YOLO model..')
    sys.stdout.flush()
    det_loader = DetectionLoader(data_loader, batchSize=args.detbatch).start()
    det_processor = DetectionProcessor(det_loader).start()
    
    # Load pose model
    pose_dataset = Mscoco()
    if args.fast_inference:
        pose_model = InferenNet_fast(4 * 1 + 1, pose_dataset)
    else:
        pose_model = InferenNet(4 * 1 + 1, pose_dataset)
    pose_model.cuda()
    pose_model.eval()

    runtime_profile = {
        'dt': [],
        'pt': [],
        'pn': []
    }

    # Data writer
    save_path = os.path.join(args.outputpath, 'AlphaPose_'+ntpath.basename(videofile).split('.')[0]+'.avi')
    writer = DataWriter(args.save_video, save_path, cv2.VideoWriter_fourcc(*'XVID'), fps, frameSize).start()

    im_names_desc =  tqdm(range(data_loader.length()))
    batchSize = args.posebatch
    for i in im_names_desc:
        start_time = getTime()
        with torch.no_grad():
            (inps, orig_img, im_name, boxes, scores, pt1, pt2) = det_processor.read()
            if orig_img is None:
                break
            if boxes is None or boxes.nelement() == 0:
                writer.save(None, None, None, None, None, orig_img, im_name.split('/')[-1])
                continue

            ckpt_time, det_time = getTime(start_time)
            runtime_profile['dt'].append(det_time)
            # Pose Estimation
            
            datalen = inps.size(0)
            leftover = 0
            if (datalen) % batchSize:
                leftover = 1
            num_batches = datalen // batchSize + leftover
            hm = []
            for j in range(num_batches):
                inps_j = inps[j*batchSize:min((j +  1)*batchSize, datalen)].cuda()
                hm_j = pose_model(inps_j)
                hm.append(hm_j)
            hm = torch.cat(hm)
            ckpt_time, pose_time = getTime(ckpt_time)
            runtime_profile['pt'].append(pose_time)

            hm = hm.cpu().data
            writer.save(boxes, scores, hm, pt1, pt2, orig_img, im_name.split('/')[-1])

            ckpt_time, post_time = getTime(ckpt_time)
            runtime_profile['pn'].append(post_time)

        if args.profile:
            # TQDM
            im_names_desc.set_description(
            'det time: {dt:.3f} | pose time: {pt:.2f} | post processing: {pn:.4f}'.format(
                dt=np.mean(runtime_profile['dt']), pt=np.mean(runtime_profile['pt']), pn=np.mean(runtime_profile['pn']))
            )

    print('===========================> Finish Model Running.')
    if (args.save_img or args.save_video) and not args.vis_fast:
        print('===========================> Rendering remaining images in the queue...')
        print('===========================> If this step takes too long, you can enable the --vis_fast flag to use fast rendering (real-time).')
    while(writer.running()):
        pass
    writer.stop()
    final_result = writer.results()
    write_json(final_result, args.outputpath)
    return True

## Alpha Pose Inference Test

In [13]:
AlphaPose_Inference("../2D to 3D Pose/Dataset/Human3.6m/Videos/S1/Videos/Directions 1.54138969.mp4")

Loading YOLO model..


/home/ivan/anaconda3/envs/yungtay/lib/python3.7/site-packages/torch/nn/functional.py:2390: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")


Loading pose model from ./models/sppe/duc_se.pth


100%|██████████| 1384/1384 [02:44<00:00,  8.40it/s]


===========================> Finish Model Running.
===========================> Rendering remaining images in the queue...
===========================> If this step takes too long, you can enable the --vis_fast flag to use fast rendering (real-time).


True

## Convert all Videos in Human3.6M Datasets

In [54]:
## Load Human3.6M Video Dataset
# Load Human3.6M Skeleton

train_2d_file = '/media/ivan/Ivan/Project Lab/THESIS_FALL_DETECTION/2D to 3D Pose/3d_pose_baseline_pytorch/data/train_2d.pth.tar'
train_2d = torch.load(train_2d_file)

train_2D_Alpha={}
for key in train_2d.keys():
    print(key)
    filename = os.path.splitext(key[2])[0]
    videopath = "../2D to 3D Pose/Dataset/Human3.6m/Videos/S"+str(key[0])+"/Videos/"+os.path.splitext(key[2])[0]+".mp4"
    print(filename)
    done_flag = AlphaPose_Inference(videopath)
    if done_flag:
        # Opening JSON file 
        f = open('examples/res/'+filename+'/alphapose-results.json',) 

        # returns JSON object as  
        data = json.load(f) 


        # Closing file 
        f.close() 

        #Normalize Alpha Pose
        full_alpha=np.asarray([])

        for frame in data:
            full_alpha = np.append(full_alpha,data_converter(frame))

        full_alpha = full_alpha.reshape(len(data), 34)
        alpha_norm = norm_alpha(full_alpha)
        train_2D_Alpha[key]=alpha_norm

    print("Alpha Pose for " +filename+ " Done")

a_file = open("Human3.6M_AlphaPose.pkl", "wb")
pickle.dump(train_2D_Alpha, a_file)
a_file.close()

(1, 'Directions', 'Directions 1.54138969.h5')
Directions 1.54138969
Loading YOLO model..


/home/ivan/anaconda3/envs/yungtay/lib/python3.7/site-packages/torch/nn/functional.py:2390: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")


Loading pose model from ./models/sppe/duc_se.pth


  0%|          | 3/1384 [00:00<02:06, 10.95it/s]


KeyboardInterrupt: 

In [70]:
import pickle

a_file = open("Human3.6M_AlphaPose.pkl", "wb")
pickle.dump(train_2D_Alpha, a_file)
a_file.close()